CLUSTERING OF NEIGHBORHOODS OF TORONTO

Installing all the required libraries to start with the project.First I had imported all the libraries but there were some errors in importing scikit. Therefore had to install all of them

In [1]:
!pip install -U numpy

!pip install -U pandas

!pip install -U scipy==1.4.1

!pip install -U scikit-learn

!pip install -U imbalanced-learn

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 15.3 MB 17.5 MB/s eta 0:00:01
ERROR: tensorflow 2.1.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.5.0 which is incompatible.
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.ut

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

The link to the table we wnat to extract is passed in an object and through request library the link is accessed through beautifulsoup and the table is extracted

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url

'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
req=requests.get(url).text
soup=BeautifulSoup(req,'html5lib')

In [5]:
table_info=[]
table=soup.find('table')
for row in table.findAll('td'):
    x={}
    if row.span.text == 'Not assigned':
        pass
    else:
        x['Postal Code']=row.p.text[:3]
        x['Borough']=(row.span.text).split('(')[0]
        x['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_info.append(x)
        
#print(table_info)
df=pd.DataFrame(table_info)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.shape

(103, 3)

The neighborhoods are then placed on the map through geocoder and folium.

In [6]:
pip install geocoder

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 98 kB 12.1 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
import geocoder # import geocoder
postal_code = df['Postal Code']

# Using while Statement
latitude = []
longitude = []
n = 0

while n < len(postal_code):
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code[n]))
    lat_lng_coords = g.latlng
    print('The geograpical coordinate of {} are {}, {}.'.format(postal_code[n], lat_lng_coords[0], lat_lng_coords[1]))
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
    n = n + 1

df['Latitude'] = latitude
df['Longitude'] = longitude

df.head()

The geograpical coordinate of M3A are 43.75245000000007, -79.32990999999998.
The geograpical coordinate of M4A are 43.73057000000006, -79.31305999999995.
The geograpical coordinate of M5A are 43.65512000000007, -79.36263999999994.
The geograpical coordinate of M6A are 43.72327000000007, -79.45041999999995.
The geograpical coordinate of M7A are 43.66253000000006, -79.39187999999996.
The geograpical coordinate of M9A are 43.662630000000036, -79.52830999999998.
The geograpical coordinate of M1B are 43.811390000000074, -79.19661999999994.
The geograpical coordinate of M3B are 43.74923000000007, -79.36185999999998.
The geograpical coordinate of M4B are 43.70718000000005, -79.31191999999999.
The geograpical coordinate of M5B are 43.65739000000008, -79.37803999999994.
The geograpical coordinate of M6B are 43.70687000000004, -79.44811999999996.
The geograpical coordinate of M9B are 43.65034000000003, -79.55361999999997.
The geograpical coordinate of M1C are 43.78574000000003, -79.1587499999999

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188


In [8]:
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |           1_llvm           5 KB  conda-forge
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    _pytorch_select-0.2        |            gpu_0           2 KB
    absl-py-0.12.0             |     pyhd8ed1ab_0          96 KB  conda-forge
    aiohttp-3.7.4              |   py37h5e8e339_0         632 KB  conda-forge
    alsa-lib-1.2.3             |       h516909a_0         560 KB  conda-forge
   

jedi-0.18.0          | 923 KB    | ##################################### | 100% 
seaborn-0.11.1       | 4 KB      | ##################################### | 100% 
pyodbc-4.0.30        | 71 KB     | ##################################### | 100% 
webencodings-0.5.1   | 12 KB     | ##################################### | 100% 
sqlite-3.35.5        | 1.4 MB    | ##################################### | 100% 
python_abi-3.7       | 4 KB      | ##################################### | 100% 
jupyter_core-4.7.1   | 72 KB     | ##################################### | 100% 
libffi-3.3           | 51 KB     | ##################################### | 100% 
grpcio-1.38.0        | 2.2 MB    | ##################################### | 100% 
markupsafe-2.0.1     | 22 KB     | ##################################### | 100% 
gmp-6.2.1            | 806 KB    | ##################################### | 100% 
entrypoints-0.3      | 8 KB      | ##################################### | 100% 
cytoolz-0.11.0       | 403 K

widgetsnbextension-3 | 1.8 MB    | ##################################### | 100% 
mock-4.0.3           | 51 KB     | ##################################### | 100% 
mistune-0.8.4        | 54 KB     | ##################################### | 100% 
ld_impl_linux-64-2.3 | 618 KB    | ##################################### | 100% 
pytables-3.6.1       | 1.5 MB    | ##################################### | 100% 
more-itertools-8.8.0 | 39 KB     | ##################################### | 100% 
libevent-2.1.10      | 1.1 MB    | ##################################### | 100% 
fsspec-2021.6.0      | 79 KB     | ##################################### | 100% 
packaging-20.9       | 35 KB     | ##################################### | 100% 
libzopfli-1.0.3      | 164 KB    | ##################################### | 100% 
libxkbcommon-1.0.3   | 581 KB    | ##################################### | 100% 
vincent-0.4.4        | 28 KB     | ##################################### | 100% 
altair-4.1.0         | 614 K

libiconv-1.16        | 1.4 MB    | ##################################### | 100% 
libopus-1.3.1        | 255 KB    | ##################################### | 100% 
pyqt-impl-5.12.3     | 5.9 MB    | ##################################### | 100% 
tensorflow-estimator | 645 KB    | ##################################### | 100% 
boto3-1.17.90        | 70 KB     | ##################################### | 100% 
h5py-3.2.1           | 1.3 MB    | ##################################### | 100% 
pandoc-2.14.0.1      | 12.0 MB   | ##################################### | 100% 
async-timeout-3.0.1  | 11 KB     | ##################################### | 100% 
imageio-2.9.0        | 3.1 MB    | ##################################### | 100% 
lerc-2.2.1           | 213 KB    | ##################################### | 100% 
libev-4.33           | 104 KB    | ##################################### | 100% 
jupyterlab_server-1. | 25 KB     | ##################################### | 100% 
yarl-1.6.3           | 141 K

chardet-4.0.0        | 204 KB    | ##################################### | 100% 
libogg-1.3.4         | 206 KB    | ##################################### | 100% 
attrs-21.2.0         | 44 KB     | ##################################### | 100% 
libxml2-2.9.12       | 772 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: | b'Exception while loading config file /var/pod/.ws/ax-ext/config/wscloud/jupyter_notebook_config.py\n    Traceback (most recent call last):\n      File "/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/traitlets/config/application.py", line 737, in _load_config_files\n        config = loader.load_config()\n      File "/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/traitlets/config/loader.py", line 616, in load_config\n        self._read_file_as_dict()\n      File "/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/traitlets/config/loader.py", line 648, in 

In [9]:
CLIENT_ID = 'KSTGWM205LYCAIS2FRDXYBO3EX2341YIMOVBIR3UKDHBIJTQ' 
CLIENT_SECRET = '3DFT205PKPR4NCESYF2PQS0EBVPOHYNSIPLQ2R4FL5HTMJWM' 
VERSION = '20180605' 
LIMIT = 100 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KSTGWM205LYCAIS2FRDXYBO3EX2341YIMOVBIR3UKDHBIJTQ
CLIENT_SECRET:3DFT205PKPR4NCESYF2PQS0EBVPOHYNSIPLQ2R4FL5HTMJWM


In [10]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude_to = location.latitude
longitude_to = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_to, longitude_to))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [11]:
map_toronto = folium.Map(location=[latitude_to, longitude_to], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

now we cluster the neighborhoods by kmeans method.
First we find all the nearby places in the neighboorhood and we reorganise the table so that it can be organised and clustered by kmeans

In [12]:
from sklearn.cluster import KMeans

In [13]:
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.75245000000007, -79.32990999999998.


In [14]:
radius = 500
LIMIT = 100
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, neighborhood_longitude, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=KSTGWM205LYCAIS2FRDXYBO3EX2341YIMOVBIR3UKDHBIJTQ&client_secret=3DFT205PKPR4NCESYF2PQS0EBVPOHYNSIPLQ2R4FL5HTMJWM&v=20180605&ll=43.75245000000007,-79.32990999999998&radius=500&limit=100'

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],latitudes=df['Latitude'],longitudes=df['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [18]:
print(toronto_venues.shape)
toronto_venues.head()

(2236, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.75245,-79.32991,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.75245,-79.32991,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.75245,-79.32991,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.73057,-79.31306,Wigmore Park,43.731023,-79.310771,Park
4,Victoria Village,43.73057,-79.31306,Memories of Africa,43.726602,-79.312427,Grocery Store


In [19]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,13,13,13,13,13,13
"Alderwood, Long Branch",4,4,4,4,4,4
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",19,19,19,19,19,19
Berczy Park,48,48,48,48,48,48
...,...,...,...,...,...,...
"Willowdale, Newtonbrook",17,17,17,17,17,17
Woburn,4,4,4,4,4,4
Woodbine Heights,19,19,19,19,19,19


In [20]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo Exhibit,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo Exhibit,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.076923,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.020833,0.0,0.020833,0.000000,...,0.0,0.041667,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,"Willowdale, Newtonbrook",0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
97,Woburn,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
98,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.052632,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
99,York Mills West,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [22]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.15
1  Hong Kong Restaurant  0.08
2       Badminton Court  0.08
3         Shopping Mall  0.08
4   Shanghai Restaurant  0.08


----Alderwood, Long Branch----
                   venue  freq
0      Convenience Store  0.25
1  Performing Arts Venue  0.25
2                    Pub  0.25
3                    Gym  0.25
4    Moroccan Restaurant  0.00


----Bayview Village----
                        venue  freq
0                       Trail  0.50
1  Construction & Landscaping  0.25
2                        Park  0.25
3   Middle Eastern Restaurant  0.00
4          Miscellaneous Shop  0.00


----Bedford Park, Lawrence Manor East----
              venue  freq
0       Coffee Shop  0.11
1    Sandwich Place  0.11
2        Restaurant  0.05
3   Thai Restaurant  0.05
4  Greek Restaurant  0.05


----Berczy Park----
            venue  freq
0    Cocktail Bar  0.08
1  Sandwich Place  0.06
2     Coffee Shop  0.06
3          Bakery  0.06

                venue  freq
0         Coffee Shop  0.25
1  Chinese Restaurant  0.25
2      Discount Store  0.25
3                Park  0.25
4       Movie Theater  0.00


----Kensington Market, Chinatown, Grange Park----
                           venue  freq
0                           Café  0.09
1  Vegetarian / Vegan Restaurant  0.04
2                    Pizza Place  0.04
3                    Gaming Cafe  0.04
4                    Art Gallery  0.04


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
                        venue  freq
0                    Bus Stop   0.4
1                 Pizza Place   0.2
2                    Bus Line   0.2
3         Arts & Crafts Store   0.2
4  Modern European Restaurant   0.0


----Lawrence Manor, Lawrence Heights----
                    venue  freq
0          Clothing Store  0.20
1              Restaurant  0.07
2        Toy / Game Store  0.04
3               Bookstore  0.04
4  Furniture / Home Store  0.04


----Lawrenc

                     venue  freq
0              Auto Garage   1.0
1              Zoo Exhibit   0.0
2      Moroccan Restaurant   0.0
3  New American Restaurant   0.0
4              Music Venue   0.0


----Willowdale South----
                  venue  freq
0           Coffee Shop  0.18
1  Fast Food Restaurant  0.09
2         Movie Theater  0.05
3              Pharmacy  0.05
4     Electronics Store  0.05


----Willowdale West----
           venue  freq
0    Pizza Place  0.17
1    Coffee Shop  0.17
2     Baby Store  0.17
3  Grocery Store  0.17
4           Park  0.17


----Willowdale, Newtonbrook----
                       venue  freq
0                       Café  0.18
1  Middle Eastern Restaurant  0.12
2          Korean Restaurant  0.12
3                Pizza Place  0.06
4           Halal Restaurant  0.06


----Woburn----
                   venue  freq
0            Coffee Shop  0.25
1  Korean BBQ Restaurant  0.25
2       Business Service  0.25
3                   Park  0.25
4              

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Hong Kong Restaurant,Badminton Court,Shopping Mall,Shanghai Restaurant,Supermarket,Sushi Restaurant,Bubble Tea Shop,Bakery,Discount Store
1,"Alderwood, Long Branch",Convenience Store,Performing Arts Venue,Pub,Gym,Moroccan Restaurant,New American Restaurant,Music Venue,Museum,Moving Target,Movie Theater
2,Bayview Village,Trail,Construction & Landscaping,Park,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Nightclub,Monument / Landmark
3,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Restaurant,Thai Restaurant,Greek Restaurant,Pub,Café,Comfort Food Restaurant,Butcher,Pharmacy
4,Berczy Park,Cocktail Bar,Sandwich Place,Coffee Shop,Bakery,Farmers Market,Seafood Restaurant,Vegetarian / Vegan Restaurant,Beer Bar,Italian Restaurant,Molecular Gastronomy Restaurant


In [25]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 1], dtype=int32)

In [26]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged['Cluster Labels'] = pd.to_numeric(toronto_merged['Cluster Labels'], errors='coerce')
toronto_merged = toronto_merged.dropna(subset=['Cluster Labels'])
print(toronto_merged[toronto_merged['Cluster Labels'].isnull()])
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged.head()

Empty DataFrame
Columns: [Postal Code, Borough, Neighborhood, Latitude, Longitude, Cluster Labels, 1st Most Common Venue, 2nd Most Common Venue, 3rd Most Common Venue, 4th Most Common Venue, 5th Most Common Venue, 6th Most Common Venue, 7th Most Common Venue, 8th Most Common Venue, 9th Most Common Venue, 10th Most Common Venue]
Index: []


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.75245,-79.32991,4,Food & Drink Shop,Fast Food Restaurant,Park,Nightclub,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark
1,M4A,North York,Victoria Village,43.73057,-79.31306,1,Park,Grocery Store,Monument / Landmark,Music Venue,Museum,Moving Target,Movie Theater,Moroccan Restaurant,Zoo Exhibit,Nightclub
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,0,Coffee Shop,Restaurant,Thrift / Vintage Store,Pub,Event Space,Greek Restaurant,Food Truck,Sandwich Place,Bakery,Thai Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042,0,Clothing Store,Restaurant,Toy / Game Store,Bookstore,Furniture / Home Store,Fast Food Restaurant,Coffee Shop,Cosmetics Shop,Mexican Restaurant,Dessert Shop
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188,0,Coffee Shop,Burrito Place,Mediterranean Restaurant,Café,Gastropub,Sushi Restaurant,Sandwich Place,Falafel Restaurant,Bar,Burger Joint


In [27]:
# create map
map_clusters = folium.Map(location=[latitude_to, longitude_to], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters